In [7]:
search_for = 100
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004389286041259766
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 225
all cases: 330988581
type: [1, 1, 1, 1, 100] 100
cases of this type: 100000000
10000 51.24287060067287
20000 48.780896277387455
30000 44.598153322391546
40000 207.91079981928655
50000 201.41137065238968
60000 190.20254746466608
70000 198.59263665995437
80000 184.46146553666355
90000 203.42736276727402
100000 196.09401400889865
110000 197.40902992610904
120000 164.66045281677938
130000 176.99415799323484
140000 184.5853374656378
150000 205.12513651404066
160000 189.9466843854109
170000 172.21560899805698
180000 188.6403600109827
190000 190.71220579172305
200000 187.04218154430214
210000 196.03952843504243
220000 196.22645074714535
230000 181.65969193392036
240000 210.0649843683813
250000 210.11481461318778
260000 214.10552054102155
270000 205.51891858795446
280000 193.9953857305376
290000 196.2674959598305
300000 193.87779688555858
310000 

3040000 48.734775741805855
3050000 49.815551480785906
3060000 48.67176029846156
3070000 200.07252474148592
3080000 187.47782170802873
3090000 148.4745130281666
3100000 181.02351671774463
3110000 174.02869763155616
3120000 191.6008133328571
3130000 163.98756225735298
3140000 184.08722563302035
3150000 171.79577409784793
3160000 155.8616065122097
3170000 167.923869555018
3180000 150.31164332539436
3190000 166.58226193185234
3200000 155.49619827563535
3210000 186.69054478934427
3220000 159.5086787168717
3230000 162.33864334793006
3240000 182.0684144390303
3250000 167.65735040195477
3260000 197.1086672805151
3270000 183.8674424208077
3280000 175.17890230437274
3290000 184.67417138871997
3300000 161.7138310650014
3310000 166.0634323441303
3320000 160.7711637406856
3330000 175.6394125710361
3340000 172.1744294554981
3350000 157.4241223937782
3360000 157.6600415223146
3370000 179.88524373802517
3380000 166.06532320023723
3390000 168.02548407579306
3400000 171.15932067025187
3410000 184.154376

6120000 150.43258706799406
6130000 176.52862285822002
6140000 173.02820449581705
6150000 179.7391255559842
6160000 158.05838053081482
6170000 175.1914113941998
6180000 145.97204200852772
6190000 169.95651167081377
6200000 164.12887446846304
6210000 145.28422153224813
6220000 142.80063563696217
6230000 179.04817098082853
6240000 176.4156999861557
6250000 190.45709020678655
6260000 171.76486973175756
6270000 183.93843583594474
6280000 154.46427441705626
6290000 185.6725361709036
6300000 171.98323460064358
6310000 153.43262445908482
6320000 164.2785957722012
6330000 162.1212397349709
6340000 161.30258290532305
6350000 147.41988662819335
6360000 165.63100277553664
6370000 166.6039725933977
6380000 146.14948746290898
6390000 187.09042361110733
6400000 188.63526498593674
6410000 162.44683031169282
6420000 165.96159509684418
6430000 152.90146672823028
6440000 161.97925306503157
6450000 152.2315929312186
6460000 154.16437693411277
6470000 164.54464406194674
6480000 198.13199358295802
6490000 2

9200000 166.86162693842707
9210000 174.7151363863479
9220000 181.0894619077892
9230000 144.10954752951397
9240000 159.5391063754793
9250000 169.916341121484
9260000 171.148102999587
9270000 155.74655937169968
9280000 166.34200451958833
9290000 165.47757382391865
9300000 178.1391757866228
9310000 158.07140074933483
9320000 187.63381911420743
9330000 168.37820128016583
9340000 162.1349841901757
9350000 162.83147771632952
9360000 163.6864766492482
9370000 155.05609970234818
9380000 165.27968978199766
9390000 163.24890798101282
9400000 158.314342077039
9410000 152.4143776159543
9420000 150.92252948919568
9430000 160.46668676073182
9440000 132.65898955223892
9450000 140.24020238610296
9460000 131.29048631660424
9470000 147.70516281679443
9480000 195.3485691554981
9490000 21.67062845939121
9500000 25.053460366145824
9510000 27.870439376489834
9520000 28.471200800124304
9530000 24.270009452818268
9540000 198.06171034540216
9550000 157.36242267264228
9560000 141.9557439125938
9570000 137.22348

12200000 179.16804867131842
12210000 164.04661784463852
12220000 154.41426858273627
12230000 165.96179960729987
12240000 166.04674075303137
12250000 178.11520220443361
12260000 147.85282816153577
12270000 155.21212265930723
12280000 168.51169407894787
12290000 163.55996527897258
12300000 169.29096803024132
12310000 168.1054905233113
12320000 175.0310459669905
12330000 174.97152101753693
12340000 167.2010449223735
12350000 151.07795309120806
12360000 151.01060861659752
12370000 119.07773497349962
12380000 153.239520790632
12390000 150.07072295892817
12400000 150.52317933302763
12410000 169.6586848322051
12420000 149.85157881233388
12430000 145.17835650805307
12440000 133.53116750850066
12450000 151.55591793411963
12460000 145.18850739075572
12470000 160.84804898790154
12480000 185.74675754191412
12490000 20.070429819348398
12500000 28.33755072384681
12510000 25.849428704591453
12520000 27.258440302277112
12530000 26.785950410427606
12540000 196.99532208605692
12550000 152.75619860057876

...

320010000 3.2028017197831073
320020000 2.125339277780341
320030000 2.87145287146312
320040000 3.272527252706945
320050000 2.0561148833182052
320060000 0.6920014507150054
320070000 2.024931229796966
320080000 3.3058306055981053
320090000 2.8327160220321552
320100000 2.041661867871324
320110000 2.4959546829421386
320120000 2.7677661733439365
320130000 2.8802775229543447
320140000 1.8947337719959152
320150000 3.1811159101109374
320160000 3.336755879325052
320170000 3.0298548745588927
320180000 1.7417044577786724
320190000 1.458653295636912
320200000 2.730716651437647
320210000 3.310146961106817
320220000 3.619722898241341
320230000 1.830061634154048
320240000 2.1352228690811526
320250000 3.57045024829189
320260000 2.1582851321775713
320270000 2.56231544983232
320280000 3.0539005664562184
320290000 2.99774095724227
320300000 2.603663837595754
320310000 0.8445480999085306
320320000 2.517357596524795
320330000 2.47296564037021
320340000 3.3597917764365475
320350000 3.0780281498722633
3203600

322840000 0.3569966461734242
322850000 0.5064086826653679
322860000 0.627849579054594
322870000 0.5357069663877354
322880000 1.271883397541139
322890000 1.6566422031605246
322900000 1.4327408824623358
322910000 1.595149380845487
322920000 1.246126496787846
322930000 0.41375922756456
322940000 0.3830746987940934
322950000 0.7649369474851291
322960000 1.5779779700815082
322970000 1.186062931275593
322980000 1.1248624893951416
322990000 1.0774049959952698
323000000 1.4016738365057044
323010000 1.4876206779475807
323020000 1.6946680023911465
323030000 1.2446093141110075
323040000 0.7975398409928083
323050000 0.8147108008086746
323060000 0.37726596270427437
323070000 0.591623918319424
323080000 0.8012037397758099
323090000 1.199548648440659
323100000 0.8866018108497857
323110000 1.2672026666656533
323120000 1.140688696198351
323130000 1.524487888021489
323140000 1.368540103757527
323150000 1.4034021631730929
323160000 1.2601973697482745
323170000 0.8962888631281456
323180000 0.3927364750348

325640000 0.5684665771092972
325650000 0.5907669314022064
325660000 0.5100175761059059
325670000 0.5948735812354684
325680000 0.5768122838053108
325690000 0.4974207361945775
325700000 0.6558185065283842
325710000 0.3655532051653862
325720000 0.663184776607818
325730000 0.7371755278836688
325740000 0.621979621013244
325750000 0.6175977566266855
325760000 0.6190593028859694
325770000 0.6554711936545967
325780000 0.47540131225955484
325790000 0.4831194481406278
325800000 0.4848401291201115
325810000 0.39764194551614257
325820000 0.38830542197781137
325830000 0.6233236725560825
325840000 0.5380043243104551
325850000 0.4450388503899243
325860000 0.349778626433293
325870000 0.36595726999899414
325880000 0.472555911479738
325890000 0.7262110884947777
325900000 0.600505016219351
325910000 0.5505885789662268
325920000 0.5427615584719976
325930000 0.42453003074255913
325940000 0.49603166786372666
325950000 0.6410214297715227
325960000 0.5705119501733781
325970000 0.5422922532662021
325980000 0.5

328310000 0.23958174351653125
328320000 0.40781180009514095
328330000 0.4015238792775406
328340000 0.28735292889634106
328350000 0.34464874889920155
328360000 0.35978328226445777
328370000 0.2485977271490826
328380000 0.25447594278832275
328390000 0.21791507510505784
328400000 0.20102139415192605
328410000 0.20201223607438804
328420000 0.11041506992716921
328430000 0.11010406981656286
328440000 0.1685352032194535
328450000 0.16008911878473228
328460000 0.17976078259499206
328470000 0.14035679568996035
328480000 0.18938775986638334
328490000 0.10714484231383933
328500000 0.10692486957377195
328510000 0.18517110518116423
328520000 0.30019492876855536
328530000 0.34909117911871274
328540000 0.27639852688415845
328550000 0.20002873639859095
best so far: 0
type: [2, 1, 5, 2, 5] 100
cases of this type: 125000
328560000 0.19937480870616436
328570000 0.15530834072913063
328580000 0.26880249280757373
328590000 0.11589051131165029
328600000 0.1003733377645678
328610000 0.10264563397366472
328620

best so far: 0
type: [4, 1, 1, 5, 5] 100
cases of this type: 78125
330370000 0.05442715962337123
330380000 0.04865063894088401
330390000 0.04448436656975746
330400000 0.042640075179417924
330410000 0.05045497949194908
330420000 0.03874973114248116
330430000 0.043412082793063585
330440000 0.047689147317051886
best so far: 0
type: [4, 1, 1, 25, 1] 100
cases of this type: 15625
330450000 0.034352228054384384
best so far: 0
type: [4, 1, 5, 1, 5] 100
cases of this type: 15625
330460000 0.027787747494339942
330470000 0.032094589710791904
best so far: 0
type: [4, 1, 5, 5, 1] 100
cases of this type: 3125
best so far: 0
type: [4, 1, 25, 1, 1] 100
cases of this type: 625
best so far: 0
type: [4, 5, 1, 1, 5] 100
cases of this type: 3125
best so far: 0
type: [4, 5, 1, 5, 1] 100
cases of this type: 625
330480000 0.029962688794136046
best so far: 0
type: [4, 5, 5, 1, 1] 100
cases of this type: 125
best so far: 0
type: [4, 25, 1, 1, 1] 100
cases of this type: 25
best so far: 0
type: [5, 1, 1, 1, 20] 